In [1]:
import os
import pandas as pd
import numpy as np
import time
from sklearn.metrics import log_loss

In [2]:
os.chdir('/aichallenge')

In [3]:
import time
t1=time.time()
train = pd.read_table('training_set.tsv')
test = pd.read_table('scoring_set.tsv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,133,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


577.0972235202789


In [4]:
train.columns

Index(['training_data.innovation_challenge_key', 'training_data.renewed_yorn',
       'training_data.instance_id', 'training_data.contract_line_id',
       'training_data.minor_line_yorn', 'training_data.major_line_instance_id',
       'training_data.installation_date',
       'training_data.product_sales_order_type',
       'training_data.product_purchase_order_type',
       'training_data.instance_status',
       ...
       'training_data.service_distributor_base_theater_name',
       'training_data.service_distributor_base_distributor_normalized_name',
       'training_data.contract_line_net_usd_amount',
       'training_data.product_net_price',
       'training_data.sales_node_renewal_rate',
       'training_data.customer_renewal_rate',
       'training_data.partner_renewal_rate',
       'training_data.product_renewal_rate',
       'training_data.service_sales_node_installed_base_sales_node_renewal_rate',
       'training_data.service_partner_installed_base_partner_renewal_rate'],


In [5]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('training_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0014040470123291016


In [6]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

12.534909963607788


In [7]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop('renewed_yorn',axis=1)
test_y=test['renewed_yorn']
test = test.drop('renewed_yorn',axis=1)
t2=time.time()
print(t2-t1)

12.023839473724365


In [8]:
#remove columns which have many null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

56.62276220321655


In [9]:
len(unclean_columns)

36

In [10]:
unclean_columns

Index(['installation_date', 'service_contract_last_renewal_date',
       'service_contract_classification', 'contract_line_duration_in_days',
       'marketing_program_name', 'service_contract_discount_percentage',
       'pos_bookings_measure_transaction_type', 'pos_adjustment_type',
       'cisco_one_opportunity_type', 'ios_software_version',
       'service_offering', 'service_program', 'service_fee_amount',
       'last_date_of_support', 'end_new_service_attachment_date',
       'end_of_service_contract_renewal_date', 'end_software_maintenance_date',
       'service_level_group', 'mapped_service_list_price',
       'mapped_service_level', 'service_group', 'service_brand_code',
       'service_product_base_service_fee_amount',
       'service_product_base_last_date_of_support',
       'service_product_base_end_new_service_attachment_date',
       'service_product_base_end_of_service_contract_renewal_date',
       'service_product_base_end_software_maintenance_date',
       'service_

In [11]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>20):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

64.0497841835022


In [12]:
# 56 messy columns are removed
len(messy_columns)

77

In [13]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

9.606790542602539


In [14]:
train.shape

(6010041, 123)

In [15]:
#Remove the primary key column for data fitting
t1=time.time()
train = train.drop('innovation_challenge_key',axis=1)
test_ids=test['innovation_challenge_key']
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

7.979274272918701


In [16]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<20):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.3302736282348633


In [17]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.603743314743042


In [18]:
train[scalcols]=train[scalcols].fillna(train[scalcols].median())
test[scalcols]=test[scalcols].fillna(test[scalcols].median())

In [19]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

8.32162094116211


In [20]:
train[scalcols].describe()

,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06,6.010041e+06
mean,-3.177913e-17,5.735377e-17,5.787455e-18,-1.343047e-17,2.407080e-18,-9.765462e-19,-3.816333e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.164629e-01,-2.451377e-01,-9.088766e-02,-2.189823e-01,-4.566387e-02,-6.938574e-02,-4.881693e-02
25%,-1.146668e-01,-2.451377e-01,-2.424868e-03,-3.552587e-02,-3.341360e-02,-6.327430e-03,-7.704820e-03
50%,-1.146668e-01,-2.451377e-01,-2.424868e-03,-3.552587e-02,-3.341360e-02,-6.327430e-03,-7.704820e-03
75%,-1.146668e-01,-1.634183e-01,-2.424868e-03,-3.552587e-02,-3.341360e-02,-6.327430e-03,-7.704820e-03
max,2.899812e+02,7.503942e+01,9.262241e+02,4.831607e+02,2.205357e+02,1.934189e+02,7.311580e+02


In [21]:
train[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,6010041,6010041,6010041,6010041
unique,3,4,6,6
top,10002,-1,6,6
freq,5316273,5985929,5248934,5945627


In [22]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

86.13638377189636


In [23]:
train.shape

(6010041, 422)

In [24]:
test.shape

(1888195, 422)

In [25]:
train.isnull().any().any()

False

In [26]:
test.isnull().any().any()

False

In [27]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

20.897080898284912


In [28]:
X_train.shape

(4026727, 422)

In [29]:
clean_col_name=[]
for col in X_train.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_train.columns = clean_col_name
clean_col_name=[]
for col in X_valid.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_valid.columns = clean_col_name

In [30]:
import gc
del dataset
del train_X
del test_X
gc.collect()

NameError: name 'train_X' is not defined

In [31]:
#Time for xgboost
import xgboost as xgb
import time
t1 = time.time()
model1 = xgb.XGBClassifier(verbose=True)
#model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)
xgb1 = model1.fit(X_train, y_train)
#xgb2 = model2.fit(X_train.iloc[0:10,], y_train[0:10])
t2 = time.time()
t2-t1

4415.33873295784

In [32]:
pred = xgb1.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))

0.5322713105133697


In [33]:
clean_col_name=[]
for col in test.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
test.columns = clean_col_name
clean_col_name=[]

In [34]:
pred=xgb1.predict_proba(test)
submit=pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('xgb_all_col_train.csv',index=False)

In [35]:
submit.shape

(1888195, 2)

In [37]:
submit.iloc[1]

INNOVATION_CHALLENGE_KEY    6.000000
RENEWAL_PROBABLIITY         0.756801
Name: 139748, dtype: float64